In [1]:
# --- Setup ---
!pip install rouge_score==0.1.2
!pip install evaluate
!pip install -U accelerate --quiet
!pip install datasets
!pip install nltk

import nltk
nltk.download("punkt")
nltk.download("punkt_tab")

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6e0460155ef79a5e511adf79e4fba1588fae24d52c3dee8dd4b883afbbfb9638
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
# --- Часть II: Загрузка и исследование датасета ---

import pandas as pd
import random
import os

# Определим колонки для статьи и резюме для XSum
ARTICLE_COLUMN = 'document'
SUMMARY_COLUMN = 'summary'

# 1. Загрузка датасетов с помощью Pandas
# Теперь мы загружаем каждый сплит отдельно
try:
    # Добавляем engine='python' и on_bad_lines='skip'
    train_df = pd.read_csv("xsum_train.csv", engine='python', on_bad_lines='skip')
    test_df = pd.read_csv("xsum_test.csv", engine='python', on_bad_lines='skip')
    validation_df = pd.read_csv("xsum_validation.csv", engine='python', on_bad_lines='skip')
    print("Датасеты xsum_train.csv, xsum_test.csv, xsum_validation.csv успешно загружены.")
except FileNotFoundError:
    print("Ошибка: Убедитесь, что файлы 'xsum_train.csv', 'xsum_test.csv' и 'xsum_validation.csv' находятся в той же директории.")
    # Создаем пустые DataFrame для избежания падения, если файлы не найдены
    train_df = pd.DataFrame(columns=[ARTICLE_COLUMN, SUMMARY_COLUMN])
    test_df = pd.DataFrame(columns=[ARTICLE_COLUMN, SUMMARY_COLUMN])
    validation_df = pd.DataFrame(columns=[ARTICLE_COLUMN, SUMMARY_COLUMN]) # Добавляем и для валидации

# 2. Сэмплирование (для уменьшения вычислительной нагрузки)
# Возьмем 100 примеров из тренировочного датасета и 50 из тестового
# (или из валидационного, если тестовый слишком мал, но в XSum они примерно одинаковы)
train_sample = train_df.sample(n=min(100, len(train_df)), random_state=42)
test_sample = test_df.sample(n=min(50, len(test_df)), random_state=42) # Используем test_df напрямую

print(f"Размер тренировочной выборки: {len(train_sample)} примеров")
print(f"Размер тестовой выборки: {len(test_sample)} примеров")

# 3. Исследование: Отображение первого примера из тренировочной выборки
print("\n--- Первый пример из тренировочной выборки ---")
first_train_example = train_sample.iloc[0]

# Используем константы для колонок
print(f"Статья ({ARTICLE_COLUMN}):\n", first_train_example[ARTICLE_COLUMN][:500], "...") # Обрежем для краткости
print(f"\nЭталонное резюме ({SUMMARY_COLUMN}):\n", first_train_example[SUMMARY_COLUMN])

# 4. Проверка данных: Вывод выборок DataFrames
print("\n--- Тренировочная выборка DataFrame (первые 5 строк) ---")
print(train_sample.head())

print("\n--- Тестовая выборка DataFrame (первые 5 строк) ---")
print(test_sample.head())

print("\nКолонки в train_sample:", train_sample.columns.tolist())

Датасеты xsum_train.csv, xsum_test.csv, xsum_validation.csv успешно загружены.
Размер тренировочной выборки: 100 примеров
Размер тестовой выборки: 50 примеров

--- Первый пример из тренировочной выборки ---
Статья (document):
 Omar Bogle, Will Grigg, Andy Kellett, Donervon Daniels and Ryan Tunnicliffe are all still out.
Barnsley's Adam Hammill (toe), Andy Yiadom (shoulder), Aidy White (pelvis) and Sessi D'Almeida (knee) are out.
Paul Heckingbottom's side are looking for back to back wins for the first time since January.
BBC Radio Manchester's Paul Rowley
"Frankly, every game now is a 'must win' for the 'Tics if they're to avoid their third relegation in five seasons.
"Saturday's stoppage-time victory over already doo ...

Эталонное резюме (summary):
 Wigan forward Nick Powell could start after three months out with a hamstring injury, after his injury-time winner against Rotherham from the bench.

--- Тренировочная выборка DataFrame (первые 5 строк) ---
                               

# Part III: Summation with T5

In [7]:
# --- Часть III: Суммаризация с T5 ---

from transformers import T5ForConditionalGeneration, AutoTokenizer
import torch
import gc # Для сборщика мусора
from tqdm.notebook import tqdm # Для прогресс-бара, полезно при долгих операциях

# Определим те же колонки, что и раньше, для удобства
ARTICLE_COLUMN = 'document'
SUMMARY_COLUMN = 'summary'

# Функция для пакетной обработки (batch processing)
# Это нужно, чтобы не загружать все данные в память GPU сразу, а обрабатывать частями
def batch_generator(data, batch_size):
    for i in range(0, len(data), batch_size):
        yield data[i:i + batch_size]

# Функция для суммаризации с T5
def summarize_with_t5(articles, model_name="t5-small", batch_size=4, max_length=150, min_length=30):
    print(f"Загрузка модели и токенизатора: {model_name}...")
    # Загружаем токенизатор и модель T5
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # Определяем устройство: GPU (cuda) если доступно, иначе CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"Модель перемещена на: {device}")

    generated_summaries = []

    # Используем tqdm для отображения прогресса
    for i, batch in enumerate(tqdm(batch_generator(articles, batch_size), total=len(articles) // batch_size + (len(articles) % batch_size > 0), desc=f"Суммаризация с {model_name}")):
        # Добавляем префикс "summarize: " согласно требованию T5
        # Обрезаем входной текст, чтобы он не был слишком длинным (обычно max_length для T5 - 512)
        inputs = tokenizer(
            ["summarize: " + article for article in batch],
            max_length=512, # Максимальная длина входной последовательности для T5
            truncation=True,
            padding="max_length", # Добавляем padding
            return_tensors="pt"
        ).to(device)

        # Генерируем резюме
        summaries = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_length=max_length,
            min_length=min_length,
            num_beams=4, # Beam search для лучшего качества
            early_stopping=True # Останавливаемся, если все beam-ы завершились
        )

        # Декодируем сгенерированные токены обратно в текст
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, clean_up_tokenization_spaces=True) for s in summaries]
        generated_summaries.extend(decoded_summaries)

        # Очищаем CUDA кэш после каждого батча
        if device.type == 'cuda':
            torch.cuda.empty_cache()
        gc.collect() # Собираем мусор

    # Окончательная очистка кэша и сбор мусора
    if device.type == 'cuda':
        torch.cuda.empty_cache()
    gc.collect()

    print(f"Суммаризация с {model_name} завершена.")
    return generated_summaries

# --- Генерация резюме для тренировочной выборки с t5-small ---
print("\n--- Генерация резюме для тренировочной выборки с t5-small ---")
# Получаем список статей из нашей тренировочной выборки
articles_to_summarize = train_sample[ARTICLE_COLUMN].tolist()

# Генерируем резюме
t5_small_summaries = summarize_with_t5(
    articles_to_summarize,
    model_name="t5-small",
    batch_size=8, # Можно увеличить или уменьшить в зависимости от доступной памяти GPU
    max_length=120, # Оптимальная длина для резюме XSum
    min_length=20
)

# --- Отображение результатов ---
print("\n--- Сравнение сгенерированных и эталонных резюме (t5-small) ---")
results_df = pd.DataFrame({
    'Original_Article': train_sample[ARTICLE_COLUMN],
    'Reference_Summary': train_sample[SUMMARY_COLUMN],
    'Generated_Summary_T5_Small': t5_small_summaries
})

# Выводим первые 5 результатов для проверки
print(results_df.head())


--- Генерация резюме для тренировочной выборки с t5-small ---
Загрузка модели и токенизатора: t5-small...


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Модель перемещена на: cpu


Суммаризация с t5-small:   0%|          | 0/13 [00:00<?, ?it/s]

Суммаризация с t5-small завершена.

--- Сравнение сгенерированных и эталонных резюме (t5-small) ---
                                         Original_Article  \
32056   Omar Bogle, Will Grigg, Andy Kellett, Donervon...   
147130  Footage posted on social media showed flames s...   
35420   Robert Stewart, 43, of Duns, made no plea or d...   
128474  Robinson, now director of rugby at Bristol, wa...   
149036  Spokesperson Merepeka Raukawa-Tait said Brown ...   

                                        Reference_Summary  \
32056   Wigan forward Nick Powell could start after th...   
147130  A large fire has ripped through a residential ...   
35420   A man has appeared in court in the Scottish Bo...   
128474  Ex-England coach Andy Robinson believes the cu...   
149036  Maori women leaders have said they support a b...   

                               Generated_Summary_T5_Small  
32056   'Tics looking for back to back wins for the fi...  
147130  authorities say civil defence workers 

In [8]:
# --- Отображение одного длинного конкретного примера ---

# Возьмем первый пример из нашего DataFrame с результатами
# Используем .iloc[0] для доступа к первой строке
first_result = results_df.iloc[0]

print("--- ПОДРОБНЫЙ АНАЛИЗ ПЕРВОГО ПРИМЕРА ---")
print("\n### Оригинальная статья (Original_Article):")
# Для длинного текста лучше использовать print, чтобы он выводился полностью
print(first_result['Original_Article'])

print("\n### Эталонное резюме (Reference_Summary):")
print(first_result['Reference_Summary'])

print("\n### Сгенерированное резюме T5-Small (Generated_Summary_T5_Small):")
print(first_result['Generated_Summary_T5_Small'])

print("\n--- КОНЕЦ ПРИМЕРА ---")

--- ПОДРОБНЫЙ АНАЛИЗ ПЕРВОГО ПРИМЕРА ---

### Оригинальная статья (Original_Article):
Omar Bogle, Will Grigg, Andy Kellett, Donervon Daniels and Ryan Tunnicliffe are all still out.
Barnsley's Adam Hammill (toe), Andy Yiadom (shoulder), Aidy White (pelvis) and Sessi D'Almeida (knee) are out.
Paul Heckingbottom's side are looking for back to back wins for the first time since January.
BBC Radio Manchester's Paul Rowley
"Frankly, every game now is a 'must win' for the 'Tics if they're to avoid their third relegation in five seasons.
"Saturday's stoppage-time victory over already doomed Rotherham United after twice going behind revived memories of the club's great survivals in their Premier League days.
"But, with most of their rivals winning, Graham Barrow's side are still seven points from safety with five matches left.
"Victory over Barnsley, promoted with Wigan from League One last season, will narrow the gap, but with away matches upcoming at promotion-chasing Brighton and Reading, th

# Part IV: Accuracy Assessment

In [9]:
# --- Часть IV: Accuracy Evaluation ---

# 1. Расчет точности
# Сравниваем сгенерированные резюме с эталонными на точное пословное совпадение.
# Важно: для этого мы используем наши сэмплированные данные, которые уже
# содержат Generated_Summary_T5_Small, Reference_Summary и Original_Article.

# Убедимся, что DataFrame `results_df` содержит нужные колонки
if 'Generated_Summary_T5_Small' in results_df.columns and 'Reference_Summary' in results_df.columns:
    # Создаем новую колонку для бинарного сравнения
    results_df['Is_Accurate'] = (results_df['Generated_Summary_T5_Small'] == results_df['Reference_Summary'])

    # Считаем процент "точных" совпадений
    accuracy = results_df['Is_Accurate'].mean() * 100

    print(f"\n--- Результаты оценки точности ---")
    print(f"Пословная 'точность' (Accuracy) для T5-small: {accuracy:.2f}%")

    # Отобразим несколько примеров, чтобы показать, где были (или не были) совпадения
    print("\nПримеры совпадений/несовпадений (Is_Accurate):")
    print(results_df[['Reference_Summary', 'Generated_Summary_T5_Small', 'Is_Accurate']].head(10))

else:
    print("Ошибка: DataFrame 'results_df' не содержит необходимых колонок для расчета точности. Убедитесь, что Часть III выполнена успешно.")

# 2. Интерпретация результата
print("\n--- Обсуждение и интерпретация точности ---")
print("Почему 'точность' (accuracy) для суммаризации, скорее всего, очень низкая или равна нулю?")
print("1. Разнообразие формулировок: Даже два человека, суммируя один и тот же текст, скорее всего, напишут разные резюме, используя разные слова и структуру предложений, но передавая схожий смысл.")
print("2. Сложность задачи: Суммаризация - это задача генерации текста, а не выбора из предопределенных категорий. Модель должна не просто угадать слово, а создать связный и осмысленный текст.")
print("3. Отсутствие синонимов: Метрика 'точности' не учитывает синонимы, перефразирование или разную структуру предложений. Если модель использует другое слово с тем же значением, это будет считаться ошибкой.")
print("\nВывод: Эта метрика почти всегда неприменима для оценки качества суммаризации, потому что она слишком строга и не отражает нюансов языка.")


--- Результаты оценки точности ---
Пословная 'точность' (Accuracy) для T5-small: 0.00%

Примеры совпадений/несовпадений (Is_Accurate):
                                        Reference_Summary  \
32056   Wigan forward Nick Powell could start after th...   
147130  A large fire has ripped through a residential ...   
35420   A man has appeared in court in the Scottish Bo...   
128474  Ex-England coach Andy Robinson believes the cu...   
149036  Maori women leaders have said they support a b...   
91082   The Department of Health has said job offers h...   
2783    A Hungarian member of the European Parliament ...   
68997   Cardiff and the surrounding area would be "ide...   
10484   A blind man was barred from using a taxi by a ...   
43416   Warren Gatland says the door is not fully clos...   

                               Generated_Summary_T5_Small  Is_Accurate  
32056   'Tics looking for back to back wins for the fi...        False  
147130  authorities say civil defence workers 

# Part V: Implementation of the ROUGE Metric

ROUGE measures the quality of a resume by comparing it to reference resumes, counting the number of matching "units" (words, word pairs, word sequences) between them.

In [10]:
# --- Часть V: ROUGE Metric Implementation ---

import evaluate
import nltk
from nltk.tokenize import sent_tokenize # Импортируем токенизатор предложений

# 1. Загрузка метрики ROUGE
print("Загрузка метрики ROUGE...")
rouge = evaluate.load("rouge")
print("Метрика ROUGE успешно загружена.")

# 2. Предобработка для ROUGE
print("\n--- Объяснение предобработки для ROUGE ---")
print("Для корректного расчета ROUGE необходимо, чтобы каждое предложение в резюме ")
print("было разделено символом новой строки (\\n). Это позволяет ROUGE более точно ")
print("сравнивать содержание, а не просто последовательность слов.")
print("Мы будем использовать 'nltk.tokenize.sent_tokenize' для разбиения текста на предложения.")
print("\nПример:")
text_example = "Это первое предложение. Это второе предложение."
tokenized_example = "\n".join(sent_tokenize(text_example))
print(f"Оригинал: '{text_example}'")
print(f"После токенизации для ROUGE:\n'{tokenized_example}'")

# 3. Определение функции compute_rouge_score
def compute_rouge_score(predictions, references):
    """
    Вычисляет ROUGE-scores для списка сгенерированных резюме по отношению к эталонным.

    Args:
        predictions (list of str): Список сгенерированных резюме.
        references (list of str): Список эталонных резюме.

    Returns:
        dict: Словарь с ROUGE-метриками (rouge1, rouge2, rougel, rougelsum).
    """
    # Предобработка: разбиваем каждое резюме на предложения и соединяем их через \n
    # Это стандартная практика для ROUGE, особенно при использовании пакетов,
    # которые ожидают такого формата для более точного соответствия
    formatted_predictions = ["\n".join(sent_tokenize(p)) for p in predictions]
    formatted_references = ["\n".join(sent_tokenize(r)) for r in references]

    # Вычисляем ROUGE-метрики
    # use_stemmer=True может улучшить совпадения за счет приведения слов к их основе
    results = rouge.compute(
        predictions=formatted_predictions,
        references=formatted_references,
        use_stemmer=True # Использование стемминга для улучшения совпадений
    )

    # Округление результатов для удобства чтения
    # results = {k: round(v * 100, 2) for k, v in results.items()}
    # Иногда результаты выдаются как объекты ScoreResult, их нужно преобразовать
    # Убедимся, что мы получаем числовые значения для округления
    rounded_results = {}
    for k, v in results.items():
        if isinstance(v, dict) and 'fmeasure' in v: # Если это dict с fmeasure, recall, precision
             rounded_results[k] = round(v['fmeasure'] * 100, 2) # Обычно интересна f-мера
        elif isinstance(v, (float, int)): # Если это просто число (например, 'mid' в старых версиях)
             rounded_results[k] = round(v * 100, 2)
        else: # На случай, если формат изменится или будет что-то еще
             rounded_results[k] = v # Оставляем как есть, если не число

    return rounded_results

print("\nФункция compute_rouge_score успешно определена.")

# --- Тестовый запуск функции compute_rouge_score ---
# Возьмем первые несколько резюме из наших сгенерированных и эталонных для теста
test_predictions = results_df['Generated_Summary_T5_Small'].tolist()[:3]
test_references = results_df['Reference_Summary'].tolist()[:3]

print("\n--- Тестовый запуск ROUGE ---")
print("Тестовые сгенерированные резюме:", test_predictions)
print("Тестовые эталонные резюме:", test_references)

sample_rouge_scores = compute_rouge_score(test_predictions, test_references)
print("\nROUGE Scores (первые 3 примера):", sample_rouge_scores)
print("\nОбратите внимание на значения ROUGE-1, ROUGE-2 и ROUGE-L. Это проценты совпадений.")

Загрузка метрики ROUGE...


Метрика ROUGE успешно загружена.

--- Объяснение предобработки для ROUGE ---
Для корректного расчета ROUGE необходимо, чтобы каждое предложение в резюме 
было разделено символом новой строки (\n). Это позволяет ROUGE более точно 
сравнивать содержание, а не просто последовательность слов.
Мы будем использовать 'nltk.tokenize.sent_tokenize' для разбиения текста на предложения.

Пример:
Оригинал: 'Это первое предложение. Это второе предложение.'
После токенизации для ROUGE:
'Это первое предложение.
Это второе предложение.'

Функция compute_rouge_score успешно определена.

--- Тестовый запуск ROUGE ---
Тестовые сгенерированные резюме: ["'Tics looking for back to back wins for the first time since January. 'Tics looking for back to back wins for the first time since January. 'Tics are seven points from safety with five games left.", 'authorities say civil defence workers "successfully evacuated" the building. it is not clear what caused the fire.', 'Robert Stewart, 43, made no plea or decl

# Part VI: Understanding ROUGE Scores

In [11]:
# --- Часть VI: Understanding ROUGE Scores ---

print("\n--- Часть VI: Понимание ROUGE Scores ---")

# 1. Тест на точное совпадение
print("\n### 1. Тест на точное совпадение (Perfect Match Test)")
# Если предсказанное резюме ИДЕНТИЧНО эталонному
pred_exact = ["Это тестовое предложение."]
ref_exact = ["Это тестовое предложение."]
print(f"Предсказание: '{pred_exact[0]}'")
print(f"Эталон:      '{ref_exact[0]}'")
exact_match_scores = compute_rouge_score(pred_exact, ref_exact)
print(f"ROUGE Scores (точное совпадение): {exact_match_scores}")
print("Ожидание: Все ROUGE-метрики должны быть близки к 100%, так как есть полное совпадение.")

# 2. Тест на нулевое предсказание (Null Prediction Test)
print("\n### 2. Тест на нулевое предсказание (Null Prediction Test)")
# Если предсказанное резюме пустое или не содержит полезной информации
pred_null = [""] # Пустое предсказание
ref_null = ["Это эталонное резюме."]
print(f"Предсказание: '{pred_null[0]}'")
print(f"Эталон:      '{ref_null[0]}'")
null_pred_scores = compute_rouge_score(pred_null, ref_null)
print(f"ROUGE Scores (пустое предсказание): {null_pred_scores}")
print("Ожидание: Все ROUGE-метрики должны быть 0%, так как нет совпадений.")

# 3. Эффект стемминга (Stemming Effect)
print("\n### 3. Эффект стемминга (Stemming Effect)")
# Продемонстрируем, как use_stemmer=True влияет на подсчет
# Пример: слова 'cat' и 'cats'
pred_stem = ["The cats are playing."]
ref_stem = ["The cat is playing."]

print(f"Предсказание: '{pred_stem[0]}'")
print(f"Эталон:      '{ref_stem[0]}'")

# Стемминг включен (как в нашей функции compute_rouge_score)
stemmed_scores = compute_rouge_score(pred_stem, ref_stem)
print(f"ROUGE Scores (со стеммингом): {stemmed_scores}")

# Для сравнения, если бы стемминг был ВЫКЛЮЧЕН (временно создадим новую функцию)
def compute_rouge_no_stem(predictions, references):
    rouge_no_stem = evaluate.load("rouge")
    formatted_predictions = ["\n".join(sent_tokenize(p)) for p in predictions]
    formatted_references = ["\n".join(sent_tokenize(r)) for r in references]
    results = rouge_no_stem.compute(predictions=formatted_predictions, references=formatted_references, use_stemmer=False)
    rounded_results = {}
    for k, v in results.items():
        if isinstance(v, dict) and 'fmeasure' in v:
             rounded_results[k] = round(v['fmeasure'] * 100, 2)
        elif isinstance(v, (float, int)):
             rounded_results[k] = round(v * 100, 2)
        else:
             rounded_results[k] = v
    return rounded_results

no_stem_scores = compute_rouge_no_stem(pred_stem, ref_stem)
print(f"ROUGE Scores (без стемминга): {no_stem_scores}")
print("Ожидание: При стемминге ROUGE-метрики должны быть выше, так как 'cat' и 'cats' будут считаться одинаковыми.")


# 4. N-gram анализ (ROUGE-1, ROUGE-2)
print("\n### 4. N-gram Analysis (ROUGE-1, ROUGE-2)")
# Как меняются ROUGE-1 и ROUGE-2 при разной степени совпадения
# Пример 1: Много общих униграмм, мало биграмм
pred_ngram_1 = ["The quick brown fox jumps."]
ref_ngram_1 = ["The fox jumps quickly."]
print(f"Предсказание: '{pred_ngram_1[0]}'")
print(f"Эталон:      '{ref_ngram_1[0]}'")
ngram_scores_1 = compute_rouge_score(pred_ngram_1, ref_ngram_1)
print(f"ROUGE Scores: {ngram_scores_1}")
print("Ожидание: ROUGE-1 выше, чем ROUGE-2, так как общих слов много, а общих пар слов меньше.")

# Пример 2: Больше общих биграмм
pred_ngram_2 = ["The quick brown fox."]
ref_ngram_2 = ["The quick brown dog."]
print(f"Предсказание: '{pred_ngram_2[0]}'")
print(f"Эталон:      '{ref_ngram_2[0]}'")
ngram_scores_2 = compute_rouge_score(pred_ngram_2, ref_ngram_2)
print(f"ROUGE Scores: {ngram_scores_2}")
print("Ожидание: ROUGE-2 будет заметен, так как есть общие пары слов ('The quick', 'quick brown').")

# 5. Симметрия ROUGE (относительно predictions и references)
print("\n### 5. Симметрия ROUGE")
print("ROUGE-метрики (f-мера) несимметричны относительно порядка predictions и references.")
print("Хотя ROUGE-L часто является симметричным, ROUGE-1 и ROUGE-2 (особенно их precision и recall) - нет.")
print("Наша функция compute_rouge_score возвращает f-меру, которая является гармоническим средним recall и precision.")
print("ROUGE-L F-мера обычно симметрична, если использовать стандартные реализации.")
print("\nПроверим F-меру для обратного порядка:")
pred_sym = ["This is a test."]
ref_sym = ["This is a test."]
scores_normal = compute_rouge_score(pred_sym, ref_sym)
scores_reversed = compute_rouge_score(ref_sym, pred_sym) # Поменяли местами
print(f"Scores (normal order): {scores_normal}")
print(f"Scores (reversed order): {scores_reversed}")
print("Ожидание: F-меры должны быть одинаковыми, так как они сбалансированы по Precision и Recall.")


--- Часть VI: Понимание ROUGE Scores ---

### 1. Тест на точное совпадение (Perfect Match Test)
Предсказание: 'Это тестовое предложение.'
Эталон:      'Это тестовое предложение.'
ROUGE Scores (точное совпадение): {'rouge1': np.float64(0.0), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0), 'rougeLsum': np.float64(0.0)}
Ожидание: Все ROUGE-метрики должны быть близки к 100%, так как есть полное совпадение.

### 2. Тест на нулевое предсказание (Null Prediction Test)
Предсказание: ''
Эталон:      'Это эталонное резюме.'
ROUGE Scores (пустое предсказание): {'rouge1': np.float64(0.0), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.0), 'rougeLsum': np.float64(0.0)}
Ожидание: Все ROUGE-метрики должны быть 0%, так как нет совпадений.

### 3. Эффект стемминга (Stemming Effect)
Предсказание: 'The cats are playing.'
Эталон:      'The cat is playing.'
ROUGE Scores (со стеммингом): {'rouge1': np.float64(75.0), 'rouge2': np.float64(33.33), 'rougeL': np.float64(75.0), 'rougeLsum': np.float64(7

# Part VII: Comparison of Small and Large Models

In [12]:
# --- Часть VII: Сравнение маленьких и больших моделей ---

# В этом разделе мы будем использовать уже определенные функции:
# - summarize_with_t5()
# - compute_rouge_score()
# - А также наши DataFrame: train_sample, test_sample, results_df
# - И константы: ARTICLE_COLUMN, SUMMARY_COLUMN

print("\n--- Генерация резюме для тестовой выборки с t5-base ---")

# Получаем список статей из нашей тестовой выборки
# (Мы будем генерировать резюме для test_sample, чтобы потом оценить их)
articles_to_summarize_test = test_sample[ARTICLE_COLUMN].tolist()

# Генерируем резюме с t5-base
# Увеличим batch_size, если позволяет GPU, для скорости
# Увеличим max_length, так как t5-base может генерировать более длинные и подробные резюме
t5_base_summaries = summarize_with_t5(
    articles_to_summarize_test,
    model_name="t5-base", # Используем t5-base
    batch_size=4,         # Возможно, придется уменьшить, если возникают ошибки памяти
    max_length=150,       # Попробуем более длинные резюме для base модели
    min_length=30
)

# Добавляем сгенерированные резюме t5-base в наш тестовый DataFrame для анализа
test_results_df = pd.DataFrame({
    'Original_Article': test_sample[ARTICLE_COLUMN],
    'Reference_Summary': test_sample[SUMMARY_COLUMN],
    'Generated_Summary_T5_Base': t5_base_summaries
})

print("\n--- Сравнение сгенерированных и эталонных резюме (t5-base, первые 5) ---")
print(test_results_df.head())


# 2. Расчет ROUGE-метрик для t5-small (для тренировочной выборки, как мы уже делали)
# Мы уже генерировали t5_small_summaries для train_sample в Части III.
# Давай пересчитаем их ROUGE-метрики для этой же train_sample.
print("\n--- Расчет ROUGE-метрик для t5-small на тренировочной выборке ---")
t5_small_rouge_scores = compute_rouge_score(
    predictions=results_df['Generated_Summary_T5_Small'].tolist(),
    references=results_df['Reference_Summary'].tolist()
)
print(f"ROUGE-метрики для t5-small: {t5_small_rouge_scores}")

# 3. Расчет ROUGE-метрик для t5-base на тестовой выборке
print("\n--- Расчет ROUGE-метрик для t5-base на тестовой выборке ---")
t5_base_rouge_scores = compute_rouge_score(
    predictions=test_results_df['Generated_Summary_T5_Base'].tolist(),
    references=test_results_df['Reference_Summary'].tolist()
)
print(f"ROUGE-метрики для t5-base: {t5_base_rouge_scores}")


# 4. Сравнение результатов
print("\n--- Сводная таблица ROUGE-метрик ---")
comparison_data = {
    'Model': ['t5-small', 't5-base'],
    'ROUGE-1': [t5_small_rouge_scores.get('rouge1', 0), t5_base_rouge_scores.get('rouge1', 0)],
    'ROUGE-2': [t5_small_rouge_scores.get('rouge2', 0), t5_base_rouge_scores.get('rouge2', 0)],
    'ROUGE-L': [t5_small_rouge_scores.get('rougeL', 0), t5_base_rouge_scores.get('rougeL', 0)],
    'ROUGE-Lsum': [t5_small_rouge_scores.get('rougeLsum', 0), t5_base_rouge_scores.get('rougeLsum', 0)]
}
comparison_df = pd.DataFrame(comparison_data)
print(comparison_df.to_string(index=False)) # to_string(index=False) для красивого вывода без индексов

print("\n--- Обсуждение сравнения моделей ---")
print("Как правило, ожидается, что модель 't5-base' покажет более высокие значения ROUGE-метрик, чем 't5-small'.")
print("Это связано с её большим размером (больше параметров) и, как следствие, большей способностью к обучению и пониманию нюансов языка.")
print("Большие модели могут лучше улавливать контекст, извлекать более релевантную информацию и генерировать более связные и точные резюме.")
print("Однако, они также требуют больше вычислительных ресурсов и времени для работы.")


--- Генерация резюме для тестовой выборки с t5-base ---
Загрузка модели и токенизатора: t5-base...


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Модель перемещена на: cpu


Суммаризация с t5-base:   0%|          | 0/13 [00:00<?, ?it/s]

Суммаризация с t5-base завершена.

--- Сравнение сгенерированных и эталонных резюме (t5-base, первые 5) ---
                                       Original_Article  \
1396  The sanctions freeze the US assets of those af...   
543   Sir Gareth attended a ceremony at Windsor Cast...   
4755  Emmanuel Manzi from Italy won the event at Lla...   
4730  Rajab, who heads the Bahrain Centre for Human ...   
5840  11 August 2015 Last updated at 19:18 BST\nSafi...   

                                      Reference_Summary  \
1396  The US government has imposed sanctions on 13 ...   
543   Wales rugby great Gareth Edwards has been knig...   
4755  About 600 runners have taken part in the annua...   
4730  Prominent Bahraini human rights activist Nabee...   
5840  An 11-year-old migrant from Afghanistan has be...   

                              Generated_Summary_T5_Base  
1396  sanctions freeze assets of those affected, sto...  
543   the 68-year-old former scrum half won 53 caps ...  
4755  Em

# SUM

The results confirm our expectations: t5-base, being a larger model, demonstrates better performance in summarizing all key ROUGE metrics compared to t5-small.